### Import libraries

In [41]:
import hdlparse.verilog_parser as vlog
import re
import random

### open and read verilog file

In [42]:
vlog_ex = vlog.VerilogExtractor()
fname = input('Enter verilog file path: ')
with open(fname, 'rt') as fh:
  code = fh.read()
vlog_mods = vlog_ex.extract_objects_from_source(code)

vlog_mods = vlog_ex.extract_objects(fname)

### print module name and ports

In [43]:
for m in vlog_mods:
  print('Module "{}":'.format(m.name))

  print('  Ports:')
  for p in m.ports:
    print('\t{:20}{:8}{}'.format(p.name, p.mode, p.data_type))

Module "Test1":
  Ports:
	HasAccess           input   wire
	Input_Pin           input   reg [3:0]
	message             output  reg [3:0]
	Welcome             output  reg


### vector_size function return vector size or 1 if it not a vector

In [44]:
def vector_size(line):
	if re.search(r'\[', line):
		lval = re.findall(r'\[(.*):', line)[0]
		rval = re.findall(r'\[.*:(.*)\]', line)[0]
		return (abs(int(rval) - int(lval)) + 1)
	return 1

### add each inputs and outputs in list as dictionary

In [45]:
for m in vlog_mods:
  inputs_list = []
  print('\n  Ports:')
  for p in m.ports:
        if (p.data_type).find('[') != -1 :
            inputs_list.append({"name":p.name ,"mode":p.mode ,"data_type":(p.data_type).split()[0],"length":vector_size((p.data_type).split()[1]) })
        else :
            inputs_list.append({"name":p.name ,"mode":p.mode ,"data_type":p.data_type, "length": 1})
        print('\t{:20}{:8}{}'.format(p.name, p.mode, p.data_type))



  Ports:
	HasAccess           input   wire
	Input_Pin           input   reg [3:0]
	message             output  reg [3:0]
	Welcome             output  reg


In [46]:
if_cases_conditions = set()
always_conditions = set()
case_conditions = set()

### Control flow parsing

#### parseing if conditions

In [47]:
def parse_if(line):
    if line.find("||") != -1 or line.find("&&") != -1:
        for port in inputs_list :
            if port["mode"]=="input" :
                if line.find(port["name"]) != -1 :
                    if_cases_conditions.add(port["name"])
    else :
        s = re.split("if\s*\(\s*!?",line)[1]
        s = re.split("\s*\)",s)[0]
        if_cases_conditions.add(s)

In [48]:
s = "if       (     HasAccess ||   b)"
print(((re.split("if\s*\(\s*!?",s))[1]).split()[0])
# parse_if(str)
# print(if_cases_conditions)

HasAccess


#### parseing always statement

In [49]:
def parse_always(line) :
    if line.find("||") != -1 or line.find("&&") != -1 or line.find("or") != -1 or line.find("and") != -1:
        for port in inputs_list :
            if port["mode"]=="input" :
                if line.find(port["name"]) != -1 :
                    always_conditions.add(port["name"])
    else :
        s = re.split("always\s*@\s*\(\s*!?",line)[1]
        s = re.split("\s*\)",s)[0]
        always_conditions.add(s)

#### parseing case statemnt

In [50]:
def parse_case(line):
    s = re.split("case\s*\(\s*",line)[1]
    s = re.split("\s*\)",s)[0]
    case_conditions.add(s)

In [51]:
file = open(fname,"rt")
for line in file :
    if re.search("if\s*\(\s*\w+",line) :
        parse_if(line)
    if re.search("always\s*@\s*\(\s*\w+\s*",line):
        parse_always(line)
    if re.search("case\s*\(\s*\w+\s*\)",line) :
        parse_case(line)
print(if_cases_conditions)
print(always_conditions)
print(case_conditions)

{'Input_Pin'}
{'HasAccess'}
{'HasAccess'}


### generate vector function generate the vector if size > 1

In [52]:
def generate_vector(port_length):
    if port_length == 1:
        return ' '
    else:
        return (' [' + str(port_length - 1) + ':0] ')

### generate range function for $urandom function in testbench

In [53]:
def generate_range(port_length):
    if port_length == 1:
        return '1'
    else:
        return str(pow(2,port_length)-1)

### add testbench content in string

In [54]:
def testbench_header_and_ports():
    content = ""
    content += ('module ' + m.name + '_tb();\n')
    for port in inputs_list:
        if port["mode"].find('input', 0, 5) != -1:
            content += ('    ' + port["mode"].replace('input', 'reg') +  generate_vector(port["length"]) + port["name"] + '_tb;\n')
        elif port["mode"].find('output', 0, 6) != -1:
            content += ('    ' + port["mode"].replace('output', 'wire') + generate_vector(port["length"]) + port["name"] +  '_tb;\n')
    return content

In [55]:
def testbench_initial_block(time_delay):
    global val
    content = ""
    content += ('  integer i;\n  begin\n//initial block\ninitial\n  begin\n    //initial values\n')
    minPortSize = 5
    for port in inputs_list :
        minPortSize = max(port["length"],minPortSize)
    iterationCounter = int(pow(2,minPortSize)/2)
    iterationCounter = min(iterationCounter, 30)
    for i in range (iterationCounter):
        if(i != 0):
            content += ('  #'+ str(time_delay)+'\n')
        for line in inputs_list:
            if line["mode"] == 'input':
                val = line["length"]
                content += ('    ' + line["name"] + '_tb = '+ str(val) + '\'b'+ bin(random.randint(0, pow(2,val)-1))[2:].zfill(val)+ ';' + '\n')

    content += ('\n//Random generated constraints\n')
    content += ('for(i = 0; i < 100; i = i+1)\n  begin\n  #'+str(time_delay)+'\n')
    for line in inputs_list:
            if line["mode"] == 'input':
                val = line["length"]
                content += ('    ' + line["name"] + '_tb = '+ '$urandom_range(0,'+generate_range(val) + ');' + '\n')
    content+= ('  end\n')
    content += ('  $finish;\nend\n')
    return content

In [56]:
def testbench_design_instance():
    content = ""
    content += ('// instaniate design instance\n  ' + m.name + ' DUT (\n')
    for i,line in enumerate(inputs_list, 0):
        content += ('    .' + (line["name"]).replace('_tb', '') + '(' + line["name"]+ '_tb'+')')
        if i != len(inputs_list)-1:
            content += (',\n')
        else:
            content += ('\n);\n')
    return content

In [57]:
def testbench_monitor_block() :
    content = ""
    word_size = []
    content += ('initial begin\n  $display ("        			","Time ')
    for port in inputs_list :
        content += (port["name"]+ '  ')
        word_size.append(len(port["name"]))
    content += ('");\n  $monitor ("     %d')
    for i in range(1, len(inputs_list)+1) :
        w = max(word_size[i-1], port["length"])
        content += ('{space:{width}}{id}'.format(space = ' ', width = w, id = '%d'))
    content += ('" , $time')
    for port in inputs_list :
        content += (','+ port["name"]+'_tb')
    content += (');\nend\nend\nendmodule')
    return content

### testbench file generation

In [58]:
def generate_output_file(file_content):
    output_file = fname.replace('.v', '_tb.v') 
    with open(output_file, 'w') as f:
            f.write(file_content)

### Testbench generator

In [59]:
time_delay = input('Enter time delay: ')
test_bench_content = ""
test_bench_content += testbench_header_and_ports() + testbench_initial_block(time_delay) + testbench_design_instance() + testbench_monitor_block()
generate_output_file(test_bench_content)